In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from google import genai
from google.generativeai import GenerativeModel

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
openai = OpenAI()
gemini = genai.Client(api_key = google_api_key)

OPENAI_MODEL = 'gpt-4o-mini'
GEMINI_MODEL = 'gemini-2.0-flash' 
OLLAMA_MODEL = 'llama3.2'

In [ ]:
system_message = "You are a helpful assistant"

In [ ]:
def chat_gpt(message, history):
    prompt = [
        {"role":"system", "content": system_message},
    ] + history + [
        {"role": "user", "content": message}
    ]

    print("History is: ", history)
    print("User message is: ", message)
    print("Prompt is: ", prompt)

    try:
        response = openai.chat.completions.create(
            model=OPENAI_MODEL,
            messages=prompt,
            stream=True
        )

        result = ""
        for chunk in response:
            result += chunk.choices[0].delta.content or ""
            yield result
    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        return None
    

In [ ]:
gr.ChatInterface(fn=chat_gpt, type="messages").launch()

In [ ]:
def chat_gemini(message, history):
    prompt = [] 
    
    for his in history:
        if his["role"] == "assistant":
            prompt += [
                {"role": "model", "parts": [{"text":his["content"]}]}
            ]
        elif his["role"] == "user":
            prompt  += [
                {"role": "user", "parts": [{"text":his["content"]}]}
            ]
    
    prompt  += [
        {"role": "user", "parts": [{"text":message}]}
    ]

    print("History is: ", history)
    print("User message is: ", message)
    print("Prompt is: ", prompt)

    try:
        model_instance = GenerativeModel(
                GEMINI_MODEL,
                system_instruction=system_message
        )

        response_stream = model_instance.generate_content(
            contents=prompt,
            stream=True
        )

        full_response = ""

        for chunk in response_stream:
            full_response += chunk.text
            yield full_response
    except Exception as e:
            print(f"An error occurred during the Gemini API call: {e}")
            return None

In [ ]:
gr.ChatInterface(fn=chat_gemini, type="messages").launch(inbrowser=True)